# Detecting COVID-19 from X-ray images using Deep Java Library

*Disclaimer: this tutorial is intended for educational purpose only. The application was developed using experimental code with a very limited COVID-19 dataset. The result should not be used for any medical diagnose for COVID-19. This content has not been reviewed or approved by any scientists or medical professionals.*

## Introduction
As you may already know, there is a on-going global pandemic caused by the [Coronavirus disease (COVID-19)](https://www.cdc.gov/coronavirus/2019-ncov/index.html).  It’s an infectious disease caused by a new virus that quickly spread over the world. It has caused severe health, financial, and economic impact globally. Many countries have declared a state of emergency and we are facing many unprecedented challenges in dealing with this disease. Testing COVID-19 is one of main challenges as we need to identify people with COVID-19 and isolate them to slow down the spread of the disease. To know more about COVID-19, please follow the [CDC website](https://www.cdc.gov/coronavirus/2019-ncov/index.html) and your local public health authorities.

In this tutorial, we will use a pretrained Keras DJL model to predict if the X-ray scanned lungs is infected or not.

## Preparation

This tutorial requires the installation of Java Kernel. To install the Java Kernel, see the [README](https://github.com/awslabs/djl/blob/master/jupyter/README.md).

These are the dependencies we will use:

In [ ]:
// %mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/

%maven ai.djl:api:0.5.0
%maven ai.djl.tensorflow:tensorflow-engine:0.5.0
%maven ai.djl.tensorflow:tensorflow-model-zoo:0.5.0
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26

// See https://github.com/awslabs/djl/blob/master/tensorflow/tensorflow-engine/README.md
// for more TensorFlow library selection options
%maven ai.djl.tensorflow:tensorflow-native-auto:2.1.0

### Import java packages

In [ ]:
import ai.djl.inference.Predictor;
import ai.djl.modality.Classifications;
import ai.djl.modality.cv.ImageFactory;
import ai.djl.modality.cv.Image;
import ai.djl.modality.cv.util.NDImageUtils;
import ai.djl.ndarray.NDArray;
import ai.djl.ndarray.NDList;
import ai.djl.repository.zoo.Criteria;
import ai.djl.repository.zoo.ModelZoo;
import ai.djl.repository.zoo.ZooModel;
import ai.djl.translate.TranslateException;
import ai.djl.translate.Translator;
import ai.djl.translate.TranslatorContext;
import ai.djl.training.util.*;
import ai.djl.util.*;
import java.net.URL;
import java.nio.file.Paths;
import java.nio.file.Files;
import java.util.Arrays;
import java.util.List;

### Download and unzip model file

In [ ]:
var destination = "build/tf_models/covid/saved_model.zip";
DownloadUtils.download("https://djl-tensorflow-javacpp.s3.amazonaws.com/tensorflow-models/covid-19/saved_model.zip", destination, new ProgressBar());
ZipUtils.unzip(Files.newInputStream(Paths.get(destination)), Paths.get("build/tf_models/covid"));

### Dive deep into Translator

To successfully run inference, we need to define some preprocessing and post processing logic to achieve the best 
prediction result and understandable output.

In [ ]:
private static final class MyTranslator implements Translator<Image, Classifications> {

    private static final List<String> CLASSES = Arrays.asList("covid-19", "normal");

    @Override
    public NDList processInput(TranslatorContext ctx, Image input) {
        NDArray array =
                input.toNDArray(
                        ctx.getNDManager(), Image.Flag.COLOR);
        array = NDImageUtils.resize(array, 224).div(255.0f);
        return new NDList(array);
    }

    @Override
    public Classifications processOutput(TranslatorContext ctx, NDList list) {
        NDArray probabilities = list.singletonOrThrow();
        return new Classifications(CLASSES, probabilities);
    }
}

As you can see above, the translator resizes the image to 224x224 and normalizes the image by dividing by 255 before feeding it into the model. When doing inference, you need to follow the same pre-processing procedure as was used during training. In this case, we need to match the Keras training code. After running prediction, the model outputs probabilities of each class as an [NDArray](https://javadoc.io/doc/ai.djl/api/latest/index.html). We need to tell the predictor to translate it back to classes, namely “covid-19” or “normal”.

Until this point, all preparation work is done, we can start working on the prediction logic.

## Predict using DJL

### Load the image
We are going to load an CT scanned image of an infected lung from internet 

In [ ]:
var imagePath = "https://github.com/ieee8023/covid-chestxray-dataset/blob/master/images/01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg?raw=true";
var image = ImageFactory.getInstance().fromUrl(new URL(imagePath));
image

### Load your model
Next, we will set the model zoo location to the `build/tf_models/covid/saved_model` directory we saved the model to. You can also create your own Repository to avoid manually managing files.

In [ ]:
System.setProperty("ai.djl.repository.zoo.location", "build/tf_models/covid/saved_model");
Criteria<Image, Classifications> criteria =
        Criteria.builder()
                .setTypes(Image.class, Classifications.class)
                .optTranslator(new MyTranslator())
                .build();
ZooModel model = ModelZoo.loadModel(criteria);

### Run inference
Lastly, we will need to create a predictor using our model and translator. Once we have a predictor, we simply need to call the predict method on our test image.

In [ ]:
Predictor<Image, Classifications> predictor = model.newPredictor();
Classifications classifications = predictor.predict(image);

classifications